
<br>
Sampling from an arbitrary hazard function<br>
Reference: https://gist.github.com/jcrudy/10481743<br>


In [9]:
import sys
print(sys.executable)

C:\Nayef\survival-analysis-notes\.venv\Scripts\python.exe


In [10]:
import numpy as np
import scipy.integrate
from matplotlib import pyplot as plt
from statsmodels.distributions import ECDF

In [11]:
class HazardSampler:
    def __init__(self, hazard, start=0.0, step=None):
        self.hazard = hazard
        if step is None:
            # heuristic for setting step size to be used in numerical inverse CDF
            # implementation
            # todo: are there better heuristics?
            h0 = hazard(0.0)
            if h0 > 0:
                step = 2.0 / hazard(0.0)
            else:
                # Reasonable default. Not efficient in some cases.
                step = 200.0 / scipy.integrate.quad(hazard, 0.0, 100.0)
        self.cumulative_hazard = CumulativeHazard(hazard)
        self.survival_function = SurvivalFunction(self.cumulative_hazard)
        self.cdf = Cdf(self.survival_function)
        self.inverse_cdf = InverseCdf(self.cdf, start=start, step=step, lower=0.0)
        self.sampler = InversionTransformSampler(self.inverse_cdf)
    def draw(self):
        return self.sampler.draw()

In [12]:
class InversionTransformSampler:
    def __init__(self, inverse_cdf):
        self.inverse_cdf = inverse_cdf
    def draw(self):
        u = np.random.uniform(0, 1)
        return self.inverse_cdf(u)

In [13]:
class CumulativeHazard:
    def __init__(self, hazard):
        self.hazard = hazard
    def __call__(self, t):
        return scipy.integrate.quad(self.hazard, 0.0, t)[0]

In [14]:
class SurvivalFunction:
    def __init__(self, cumulative_hazard):
        self.cumulative_hazard = cumulative_hazard
    def __call__(self, t):
        return np.exp(-self.cumulative_hazard(t))

In [15]:
class Cdf:
    def __init__(self, survival_function):
        self.survival_function = survival_function
    def __call__(self, t):
        return 1.0 - self.survival_function(t)

In [16]:
class InverseCdf:
    def __init__(
        self, cdf, start, step, precision=1e-8, lower=float("-inf"), upper=float("inf")
    ):
        self.cdf = cdf
        self.precision = precision
        self.start = start
        self.step = step
        self.lower = lower
        self.upper = upper
    def __call__(self, target_p):
        """
        Takes a proportion p (y-axis value of the cdf) and returns the corresponding
        x value from the cdf (to within a certain precision level).
        """
        last_diff = None
        step = self.step
        current_x = self.start
        while True:
            current_cdf_value = self.cdf(current_x)
            diff = current_cdf_value - target_p
            if abs(diff) < self.precision:
                break
            elif diff < 0:
                # current_x is too far to the left, so we increase it
                current_x = min(current_x + step, self.upper)
                if last_diff is not None and last_diff > 0:
                    # if diff and last_diff are opposite signs, take smaller
                    # optimization steps
                    step *= 0.5
                last_diff = diff
            else:
                # current_x is too far to the right, so we decrease it
                current_x = max(current_x - step, self.lower)
                if last_diff is not None and last_diff < 0:
                    # if diff and last_diff are opposite signs, take smaller
                    # optimization steps
                    step *= 0.5
                last_diff = diff
        return current_x